## make adjacency matrix to export

In [2]:
from pathlib import Path
import os
import numpy as np
import networkx as nx

btw, pathlib.Path is amazing, just started playing with it, fully object-oriented path manipulation

In [10]:
p = Path('DK_res-1x1x1')

In [15]:
# making sure I know what iterdir() does
[i for i in p.iterdir()][:2]

[PosixPath('DK_res-1x1x1/sub-NDARFR109LKT_acq-64dir_dwi_DK_res-1x1x1_measure-spatial-ds._elist.csv'),
 PosixPath('DK_res-1x1x1/sub-NDARCT933HF2_acq-64dir_dwi_DK_res-1x1x1_measure-spatial-ds._elist.csv')]

### check nodelist

### grab graphs

In [17]:
# list of numpy matrices
graphs = [nx.to_numpy_matrix(nx.read_weighted_edgelist(i, delimiter=',')) 
          for i in p.iterdir()]

### remove diagonals

### consider LLC somehow